In [1]:
from hyperopt import fmin, tpe, hp, partial
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, zero_one_loss

import xgboost as xgb
import pandas as pd

def GetNewDataByPandas():
    wine = pd.read_csv("/home/fonttian/Data/UCI/wine/wine.csv")
    wine['alcohol**2'] = pow(wine["alcohol"], 2)
    wine['volatileAcidity*alcohol'] = wine["alcohol"] * wine['volatile acidity']
    y = np.array(wine.quality)
    X = np.array(wine.drop("quality", axis=1))
    # X = np.array(wine)

    columns = np.array(wine.columns)

    return X, y, columns

In [2]:
# Read wine quality data from file
X, y, wineNames = GetNewDataByPandas()

# split data to [[0.8,0.2],01]
x_train_all, x_predict, y_train_all, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)

x_train, x_test, y_train, y_test = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=100)

In [3]:
dtrain = xgb.DMatrix(data=x_train,label=y_train,missing=-999.0)
dtest = xgb.DMatrix(data=x_test,label=y_test,missing=-999.0)

evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [4]:
def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['n_estimators'] = argsDict['n_estimators'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["subsample"] = argsDict["subsample"] * 0.1 + 0.5
    argsDict["min_child_weight"] = argsDict["min_child_weight"] + 1
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

In [13]:
def xgboost_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)
    
    params = {'nthread': -1,  # 进程数
              'max_depth': argsDict['max_depth'],  # 最大深度
              'n_estimators': argsDict['n_estimators'],  # 树的数量
              'eta': argsDict['learning_rate'],  # 学习率
              'subsample': argsDict['subsample'],  # 采样数
              'min_child_weight': argsDict['min_child_weight'],  # 终点节点最小样本占比的和
              'objective': 'reg:linear',
              'silent': 0,  # 是否显示
              'gamma': 0,  # 是否后剪枝
              'colsample_bytree': 0.7,  # 样本列采样
              'alpha': 0,  # L1 正则化
              'lambda': 0,  # L2 正则化
              'scale_pos_weight': 0,  # 取值>0时,在数据不平衡时有助于收敛
              'seed': 100,  # 随机种子
              'missing': -999,  # 填充缺失值
              }
    params['eval_metric'] = ['rmse']

    xrf = xgb.train(params, dtrain, 300, evallist,early_stopping_rounds=100)

    return get_tranformer_score(xrf)

In [6]:
def get_tranformer_score(tranformer):
    
    xrf = tranformer
    dpredict = xgb.DMatrix(x_predict)
    prediction = xrf.predict(dpredict, ntree_limit=xrf.best_ntree_limit)
  
    return mean_squared_error(y_predict, prediction)

In [7]:
# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         "n_estimators": hp.randint("n_estimators", 300),
         "learning_rate": hp.randint("learning_rate", 6),
         "subsample": hp.randint("subsample", 5),
         "min_child_weight": hp.randint("min_child_weight", 6),
         }

In [14]:
# 开始使用hyperopt进行自动调参
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(xgboost_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=7
[0]	eval-rmse:4.94443	train-rmse:4.81349
Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.

Will train until train-rmse hasn't improved in 100 rounds.
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=7
[1]	eval-rmse:4.60501	train-rmse:4.48118
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=7
[2]	eval-rmse:4.2925	train-rmse:4.1718
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=7
[3]	eval-rmse:4.00235	train-rmse:3.88337
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=7
[4]	eval-rmse:3.73454	train-rmse:3.61847
[14:51:10] /workspace/src/tree/

[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=7
[49]	eval-rmse:0.615696	train-rmse:0.349783
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=7
[50]	eval-rmse:0.612203	train-rmse:0.343364
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 0 pruned nodes, max_depth=7
[51]	eval-rmse:0.609981	train-rmse:0.337212
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=7
[52]	eval-rmse:0.608559	train-rmse:0.331314
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=7
[53]	eval-rmse:0.606013	train-rmse:0.325558
[14:51:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 0 pruned nodes, max_depth=7
[54]	eval-rmse:0.604564	train-rmse:0.32040

[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=7
[99]	eval-rmse:0.584621	train-rmse:0.216388
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=7
[100]	eval-rmse:0.584178	train-rmse:0.214118
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=7
[101]	eval-rmse:0.584867	train-rmse:0.212027
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=7
[102]	eval-rmse:0.584967	train-rmse:0.209889
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=7
[103]	eval-rmse:0.584245	train-rmse:0.20841
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=7
[104]	eval-rmse:0.584814	train-rmse:0

[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=7
[148]	eval-rmse:0.581068	train-rmse:0.140095
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=7
[149]	eval-rmse:0.581176	train-rmse:0.139843
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 0 pruned nodes, max_depth=7
[150]	eval-rmse:0.580919	train-rmse:0.138889
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 0 pruned nodes, max_depth=7
[151]	eval-rmse:0.581259	train-rmse:0.137622
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=7
[152]	eval-rmse:0.581291	train-rmse:0.137132
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=7
[153]	eval-rmse:0.581235	train-rmse:

[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=7
[197]	eval-rmse:0.581941	train-rmse:0.091756
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=7
[198]	eval-rmse:0.582238	train-rmse:0.090892
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=7
[199]	eval-rmse:0.582272	train-rmse:0.090529
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=7
[200]	eval-rmse:0.582338	train-rmse:0.09008
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=7
[201]	eval-rmse:0.582216	train-rmse:0.089278
[14:51:11] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=7
[202]	eval-rmse:0.582128	train-rmse:0

...

[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 188 extra nodes, 0 pruned nodes, max_depth=16
[83]	eval-rmse:0.577094	train-rmse:0.023195
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 226 extra nodes, 0 pruned nodes, max_depth=16
[84]	eval-rmse:0.577044	train-rmse:0.022465
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=16
[85]	eval-rmse:0.577081	train-rmse:0.021919
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=16
[86]	eval-rmse:0.576998	train-rmse:0.021493
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 234 extra nodes, 0 pruned nodes, max_depth=16
[87]	eval-rmse:0.577126	train-rmse:0.020776
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 0 pruned nodes, max_depth=16
[88]	eval-rmse:0.577222	train-

[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 240 extra nodes, 0 pruned nodes, max_depth=16
[132]	eval-rmse:0.577983	train-rmse:0.007709
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=16
[133]	eval-rmse:0.577964	train-rmse:0.00749
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=16
[134]	eval-rmse:0.577981	train-rmse:0.007397
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 274 extra nodes, 0 pruned nodes, max_depth=16
[135]	eval-rmse:0.577937	train-rmse:0.007184
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=16
[136]	eval-rmse:0.577967	train-rmse:0.007032
[14:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 0 pruned nodes, max_depth=16
[137]	eval-rmse:0.577979	tr

[180]	eval-rmse:0.578288	train-rmse:0.00295
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=16
[181]	eval-rmse:0.578292	train-rmse:0.002912
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=16
[182]	eval-rmse:0.578279	train-rmse:0.00286
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=16
[183]	eval-rmse:0.578271	train-rmse:0.002812
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 0 pruned nodes, max_depth=16
[184]	eval-rmse:0.578281	train-rmse:0.002754
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 130 extra nodes, 0 pruned nodes, max_depth=16
[185]	eval-rmse:0.57829	train-rmse:0.002708
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned node

[229]	eval-rmse:0.578471	train-rmse:0.001341
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=16
[230]	eval-rmse:0.578487	train-rmse:0.001325
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=16
[231]	eval-rmse:0.578494	train-rmse:0.001306
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 0 pruned nodes, max_depth=16
[232]	eval-rmse:0.578492	train-rmse:0.001292
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 134 extra nodes, 0 pruned nodes, max_depth=16
[233]	eval-rmse:0.57849	train-rmse:0.00127
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=16
[234]	eval-rmse:0.578497	train-rmse:0.001252
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned node

[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=16
[279]	eval-rmse:0.57854	train-rmse:0.000918
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[280]	eval-rmse:0.57854	train-rmse:0.000916
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=5
[281]	eval-rmse:0.578537	train-rmse:0.000914
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[282]	eval-rmse:0.57854	train-rmse:0.000914
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=8
[283]	eval-rmse:0.578537	train-rmse:0.000912
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=6
[284]	eval-rmse:0.578536	train-rmse:0.0

In [15]:
# 展示结果
RMSE = xgboost_factory(best)
# print('mse of the best xgboost:', RMSE)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best xgboost:', np.sqrt(RMSE))

[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 216 extra nodes, 0 pruned nodes, max_depth=10
[0]	eval-rmse:4.52515	train-rmse:4.40362
Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.

Will train until train-rmse hasn't improved in 100 rounds.
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 234 extra nodes, 0 pruned nodes, max_depth=10
[1]	eval-rmse:3.86168	train-rmse:3.74942
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 220 extra nodes, 0 pruned nodes, max_depth=10
[2]	eval-rmse:3.31096	train-rmse:3.19818
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 0 pruned nodes, max_depth=10
[3]	eval-rmse:2.83924	train-rmse:2.72638
[14:51:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 212 extra nodes, 0 pruned nodes, max_depth=10
[4]	eval-rmse:2.44121	train-rmse:2.32661
[14:51:46] /workspace/sr

[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=10
[48]	eval-rmse:0.570958	train-rmse:0.125113
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=10
[49]	eval-rmse:0.570175	train-rmse:0.120475
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=10
[50]	eval-rmse:0.569472	train-rmse:0.117425
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=10
[51]	eval-rmse:0.569819	train-rmse:0.114769
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 0 pruned nodes, max_depth=10
[52]	eval-rmse:0.569937	train-rmse:0.113171
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=10
[53]	eval-rmse:0.569821	train-rms

[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=10
[97]	eval-rmse:0.573987	train-rmse:0.037138
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=10
[98]	eval-rmse:0.574424	train-rmse:0.035909
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 0 pruned nodes, max_depth=10
[99]	eval-rmse:0.574743	train-rmse:0.034895
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 0 pruned nodes, max_depth=10
[100]	eval-rmse:0.574905	train-rmse:0.03386
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=10
[101]	eval-rmse:0.574928	train-rmse:0.032886
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 0 pruned nodes, max_depth=10
[102]	eval-rmse:0.575003	train

[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 224 extra nodes, 0 pruned nodes, max_depth=10
[146]	eval-rmse:0.5757	train-rmse:0.011898
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=10
[147]	eval-rmse:0.575698	train-rmse:0.01158
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 214 extra nodes, 0 pruned nodes, max_depth=10
[148]	eval-rmse:0.575802	train-rmse:0.011111
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 0 pruned nodes, max_depth=10
[149]	eval-rmse:0.57578	train-rmse:0.010759
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=10
[150]	eval-rmse:0.575778	train-rmse:0.010595
[14:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=10
[151]	eval-rmse:0.575734	trai

[194]	eval-rmse:0.576067	train-rmse:0.003558
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 0 pruned nodes, max_depth=10
[195]	eval-rmse:0.576052	train-rmse:0.003462
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=10
[196]	eval-rmse:0.576055	train-rmse:0.003421
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=10
[197]	eval-rmse:0.576046	train-rmse:0.003382
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 0 pruned nodes, max_depth=10
[198]	eval-rmse:0.576061	train-rmse:0.003308
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 0 pruned nodes, max_depth=10
[199]	eval-rmse:0.576052	train-rmse:0.003266
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nod

[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 190 extra nodes, 0 pruned nodes, max_depth=10
[244]	eval-rmse:0.5762	train-rmse:0.001405
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=10
[245]	eval-rmse:0.576209	train-rmse:0.001374
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=10
[246]	eval-rmse:0.576212	train-rmse:0.001355
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=10
[247]	eval-rmse:0.576215	train-rmse:0.00133
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=10
[248]	eval-rmse:0.57622	train-rmse:0.001316
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=10
[249]	eval-rmse:0.576222	train-r

[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=10
[293]	eval-rmse:0.576254	train-rmse:0.000857
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=10
[294]	eval-rmse:0.576253	train-rmse:0.000841
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[295]	eval-rmse:0.576252	train-rmse:0.000841
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3
[296]	eval-rmse:0.576253	train-rmse:0.00084
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[297]	eval-rmse:0.576252	train-rmse:0.00084
[14:51:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[298]	eval-rmse:0.576252	train-rmse:0.000

In [16]:
"""
best : {'learning_rate': 0.15000000000000002, 'max_depth': 10, 'min_child_weight': 6, 'n_estimators': 184, 'subsample': 0.9}
best param after transform :
{'learning_rate': 0.053000000000000005, 'max_depth': 15, 'min_child_weight': 7, 'n_estimators': 334, 'subsample': 0.59}
rmse of the best xgboost: 0.5402638952149448
"""

"\nbest : {'colsample_bytree': 0.7, 'gamma': 0.0, 'learning_rate': 0.15000000000000002, 'max_depth': 16, 'min_child_weight': 1, 'n_estimators': 440, 'subsample': 0.8}\nbest param after transform :\n{'colsample_bytree': 0.57, 'gamma': 0.0, 'learning_rate': 0.053000000000000005, 'max_depth': 21, 'min_child_weight': 2, 'n_estimators': 590, 'subsample': 0.5800000000000001}\nrmse of the best xgboost: 0.5602862515362897\n"